In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("DDoS.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225745 entries, 0 to 225744
Data columns (total 85 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Flow ID                       225745 non-null  object 
 1    Source IP                    225745 non-null  object 
 2    Source Port                  225745 non-null  int64  
 3    Destination IP               225745 non-null  object 
 4    Destination Port             225745 non-null  int64  
 5    Protocol                     225745 non-null  int64  
 6    Timestamp                    225745 non-null  object 
 7    Flow Duration                225745 non-null  int64  
 8    Total Fwd Packets            225745 non-null  int64  
 9    Total Backward Packets       225745 non-null  int64  
 10  Total Length of Fwd Packets   225745 non-null  int64  
 11   Total Length of Bwd Packets  225745 non-null  int64  
 12   Fwd Packet Length Max        225745 non-nul

In [4]:
def ip_col_classiffication(df, colum):
    calsif = []
    for i in range(len(df[colum])):

        nums = data[colum][i].split(".")
        nums = [int(num) for num in nums]

        if nums[0] == 172 and nums[1] >= 16 and nums[1] <= 31:
            calsif.append(1)
        elif nums[0] == 192 and nums[1] == 168:
            calsif.append(1)
        else:
            calsif.append(0)
        
    df[colum] = calsif
    return df

In [5]:
data = ip_col_classiffication(data, " Source IP")
data = ip_col_classiffication(data, " Destination IP")

In [6]:
data[" Source IP"].value_counts()

 Source IP
1    217082
0      8663
Name: count, dtype: int64

In [7]:
data[" Destination IP"].value_counts()

 Destination IP
1    202455
0     23290
Name: count, dtype: int64

In [8]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, drop='first')
    
data[" Label"] = ohe.fit_transform(data[[" Label"]])

c:\Users\acasalro\AppData\Local\miniconda3\envs\ia\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
data = data.select_dtypes(exclude=['object'])

In [10]:
correlaciones = data.corr()

In [11]:
label_cor = correlaciones[" Label"]
label_cor = pd.DataFrame(label_cor)

lower_bound = -0.15
upper_bound = 0.15

filtered_df = label_cor[(label_cor[' Label'] < lower_bound) | (label_cor[' Label'] > upper_bound)]

filtered_df = filtered_df[" Label"].reset_index()

In [12]:
final = pd.DataFrame()

for val in filtered_df["index"]:
    final[val] = data[val]

In [13]:
final.to_csv('Clean_DDoS.csv', index=False)